In [1]:
import json

# Read data from article_2020.json
#with open('article_2020.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Select the first 1000 entries
first_1000_entries = data[:1000]

# Write the first 1000 entries to first1000.json
with open('first1000.json', 'w', encoding='utf-8') as f:
    json.dump(first_1000_entries, f, ensure_ascii=False, indent=4)


# Read in first 1000 entries

In [21]:
import json

with open('first1000.json', 'r') as file:
    documents = json.load(file)


# preprocess data 

pls be aware that the pmid will now be used as id in the preprocessed file

In [22]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Initialize NLTK tools
stop_words = set(stopwords.words('english'))

def preprocess(text):
    """
    Tokenize, lowercase, remove stop words, and keep alphanumeric tokens for abstract and title.
    """
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Filter
    return filtered_tokens

def preprocess_mesh_major(mesh_major):
    """
    Lowercase and tokenize meshMajor terms. Keep them fairly original since they are specific terms.
    """
    return [term.lower() for term in mesh_major]



# Apply preprocessing to include abstract, title, meshMajor, and journal
preprocessed_documents = []
for doc in documents:
    preprocessed_doc = {
        "id": doc["pmid"],  # Using pmid as a unique identifier
        "tokens": preprocess(doc["abstractText"]) + preprocess(doc["title"])
                   + preprocess_mesh_major(doc["meshMajor"]),
        "journal": doc["journal"].lower(),  # Lowercase journal
        "year": doc["year"],  # Keep year as is
        "meshMajor": doc["meshMajor"]  # Keep original meshMajor for tagging or categorization
    }
    preprocessed_documents.append(preprocessed_doc)


[nltk_data] Downloading package punkt to /home/sam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
for entry in preprocessed_documents:
    print(entry)

{'id': '34823483', 'tokens': ['background', 'worldwide', 'hypertensive', 'disorders', 'pregnancy', 'hdp', 'fetal', 'growth', 'restriction', 'fgr', 'preterm', 'birth', 'remain', 'leading', 'causes', 'maternal', 'fetal', 'mortality', 'morbidity', 'fetal', 'cardiac', 'deformation', 'changes', 'first', 'sign', 'placental', 'dysfunction', 'associated', 'hdp', 'fgr', 'preterm', 'birth', 'addition', 'preterm', 'birth', 'likely', 'associated', 'changes', 'electrical', 'activity', 'across', 'uterine', 'muscle', 'therefore', 'fetal', 'cardiac', 'function', 'uterine', 'activity', 'used', 'early', 'detection', 'complications', 'pregnancy', 'fetal', 'cardiac', 'function', 'uterine', 'activity', 'assessed', 'echocardiography', 'fetal', 'electrocardiography', 'electrohysterography', 'ehg', 'study', 'aims', 'generate', 'reference', 'values', 'ehg', 'parameters', 'second', 'trimester', 'pregnancy', 'investigate', 'diagnostic', 'potential', 'parameters', 'early', 'detection', 'hdp', 'fgr', 'preterm', 'l

In [6]:
#save preprocessed docs

#with open('preprocessed_first1000.json', 'w') as file:
    json.dump(preprocessed_documents, file)


# now lets import the file again

In [1]:
import json

with open('preprocessed_first1000.json', 'r') as file:
    preprocessed_documents = json.load(file)


# import bm25 algorithm and langchain retriever

In [2]:
#from langchain_community.retrievers import BM25Retriever unused for now

from rank_bm25 import BM25Okapi


implement sample retriever

In [27]:
from rank_bm25 import BM25Okapi

# Renaming the class to avoid conflict with the LangChain's BM25Retriever
class MyBM25Retriever:
    def __init__(self, documents):
        self.documents = documents
        self.corpus = [doc['tokens'] for doc in documents]  # Assuming 'tokens' is your preprocessed text
        self.bm25 = BM25Okapi(self.corpus)

    def retrieve(self, query, top_n=10):
        query_tokens = preprocess(query)#query.split()  # Simple tokenization, replace with your preprocess method if necessary
        doc_scores = self.bm25.get_scores(query_tokens)
        print(doc_scores)
        print(query_tokens)
        top_doc_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_n]
        return [self.documents[i] for i in top_doc_indices]


In [28]:
bm25_retriever = MyBM25Retriever(preprocessed_documents)


# test retriever

In [37]:
query = "     "
top_documents = bm25_retriever.retrieve(query, top_n=5)

# Example: print the retrieved document titles or IDs
for doc in top_documents:
    print(doc['id'])  # Assuming each document has an 'id' field


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
# since this works okay but not really well i now try to implement the out of the box solution offered by langchain

In [45]:
import json
#from langchain.core.documents import Document
from langchain.docstore.document import Document
from langchain_community.retrievers import BM25Retriever

# Step 1: Load the JSON file
with open('preprocessed_first1000.json', 'r') as file:
    data = json.load(file)



In [48]:

# Step 2: Parse the JSON to extract documents
# Assuming your JSON is a list of texts
texts = [' '.join(item['tokens']) if isinstance(item['tokens'], list) else item['tokens'] for item in data]

# Step 3: Create Document objects
documents = [Document(page_content=text) for text in texts]

# Step 4: Instantiate the BM25Retriever
retriever = BM25Retriever.from_documents(documents)



In [52]:
# Step 5: Use the Retriever to get relevant documents
query = " what is as a novel candidate for treating COVID-19 via heme oxygenase-1 induction."  # Replace with your actual query
result = retriever.get_relevant_documents(query)

print(result)

[Document(page_content='severe acute respiratory syndrome coronavirus 2 causative agent coronavirus 143 million cases reported date global death rate currently licensed therapeutics controlling infection antiviral effects heme cytoprotective enzyme inhibits inflammatory response reduces oxidative stress investigated several viral infections confirm whether suppresses infection assessed antiviral activity hemin effective safe inducer infection found treatment hemin efficiently suppressed replication selectivity index besides transient expression using expression vector also suppressed growth virus cells free iron biliverdin metabolic byproducts heme catalysis also suppressed viral infection additionally hemin indirectly increased expression proteins known restrict replication overall findings suggested induced hemin effectively suppressed vitro therefore could potential therapeutic candidate hemin novel candidate treating via heme induction animals antiviral agents covid-19 cell surviva